# qe_workflow.ipynb
### Kat Nykiel, Alejandro Strachan

This notebook provides a sample workflow for running quantum espresso calculations on nanoHUB. In this example, we will relax a MXene, Ti$_3$C$_2$ using quantum espresso

**Note**: this notebook assumes you're vaguely familiar with python and Jupyter notebooks. If you aren't , please reach out to one of us and we'll be able to help



To start, let's import the python libraries we'll be using throughout the notebook, and an addition script with helper functions.

There will likely be a warning printed: this is fine. Not ideal, but it won't break the notebook

In [ ]:
# Import libraries
import numpy as np
from mp_api.client import MPRester
from pymatgen.io.pwscf import PWInput

# Import helper functions
from qe_functions import *

## Query structure from Materials Project

The first step of our workflow is choosing which system we want to simulate. [Materials Project](https://materialsproject.org/) is an open database which catalogues information on tens of thousands of materials and their properties.

If you don't have one already, now would be a good time to obtain an API key, which we'll use to connect to Materials Project. You can do so [here](https://materialsproject.org/api)

Next, we want to choose our system of interest on Materials Project. Here, we're using Ti$_3$C$_2$ on Materials Project.


<div>
<img src="images/MP_dashboard.png" width="800"/>
</div>


**task:** Run the following cells to load your key and query for your specific mp-ID

In [ ]:
# Load (or enter when prompted) your API key 
key = read_key()

In [ ]:
# Query using new Materials Project API for a specific ID
with MPRester(key) as m:
    data = m.materials.summary.search(material_ids=["mp-1094034"]) # Change this to your chosen mp-ID

This returns a lot of information, but right now we're just interested in the **structure** object. This is the Materials Project-preferred way to pass unit cell data (lattice, basis, etc.) in python

In [ ]:
struct = data[0].structure
display(struct)

## Run density functional theory with quantum espresso

Next, we want to run ionic and  variable cell relaxations using density functional theory.

We'll be doing this using [**quantum espresso**](https://www.quantum-espresso.org/) (QE), an open-source code for DFT. To make it easier to create input files, we're going to continue using *pymatgen*, a python library for computational materials science

In this example, we'll use a set of project-augmented wave (PAW) pseudopotentials with a Perdew–Burke–Ernzerhof (PBE) exchange-correlation functional. Several other choices are found [here](https://www.quantum-espresso.org/pseudopotentials/) 

### Generate QE input files using pymatgen
These two functions will let us create and run QE simulations from a Jupyter notebook, which makes it much easier to automate than manually editing the files. 

In [ ]:
def make_sim(name,struct,**kwargs):
    """
    Generate quantum espresso input files using pymatgen's PWInput class
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    **kwargs:
        dictionaries to input to pymatgen's PWInput object
    """
    # Prepare dict of pseudopotentials (i.e. {'Mg': 'Mg.upf', 'O': 'O.upf'})
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_dict = dict(zip(elements,[f"{element}.upf" for element in elements]))

    # Define input set
    input_set = PWInput(structure=struct,
                        pseudo=pseudo_dict,
                        **kwargs) # dictionaries corresponding to blocks in QE input files

    input_set.write_file(filename=f'{name}.in')
    
def run_sim(name,struct):
    """
    Submit quantum espresso runs to HPC clusters on nanoHUB
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    """
    # Write input and output files
    input_file = open(f'{name}.in','a')
    input_file.close()

    output_file = open(f'{name}.out', 'w')
    output_file.close()
    
    # Set up commands and files
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_arg = "".join([f"-i ./pseudo/pseudo_PAW/{element}.upf " for element in elements])
    COMMAND = f"espresso-6.8_pw > {output_file.name}"
    
    # Run simulation (1 node, 1 hour walltime)
    !submit -n 1 -w '01:00:00' -e QE_DISABLE_GGA_PBE=0 --runName {name} {COMMAND} {pseudo_arg} -i {input_file.name}


*Note*: this submits jobs remotely to Brown, which is being decommisioned on November 8, 2023. See the README for instructions on submitting locally.

### Run an ionic relaxation of our structure
Let's use our two functions to run an ionic relaxation in quantum espresso. The parameters of the simulation are controlled via tags, which are found [here](https://www.quantum-espresso.org/Doc/INPUT_PW.html). They are controlled by blocks (control, system, etc.) and passed to our function as dictionaries of tags

**** The kinetic energy cutoff (ecutwfc) and kpoints (kpoints_grid) have a significant effect on the convergence of the simulation. In DFT, we typically hold one parameter at a high value and vary the other to determine what minimum is necessary for convergence with respect to some property (i.e. lattice parameter).

In [ ]:
# Create an ionic relaxation sim
make_sim("relax", struct,
         control={'pseudo_dir':'./',
                  'calculation':'relax',
                  'outdir':'./',
                  'tstress':True},
         system={'ecutwfc':50},
         kpoints_grid=[3,3,3])

The input file should be made! Check your directory for a *relax.in* file. This is what quantum espresso uses to determine which simulation to run.

In [ ]:
# Run relax simulation
run_sim("relax", struct)

This simulation should take ~30 minutes to run. If it takes longer, consider lowering the kinetic energy cutoff or kpoint size, or choosing a smaller system. 

Once it's done, run the following cells to extract some outputs

In [ ]:
# Extract outputs using helper function
relax_dict = get_qe_outputs('relax.stdout')
relaxed_struct = relax_dict['structures']

In [ ]:
# Get a list of all the extracted outputs
[print(k) for k,v in relax_dict.items()];

In [ ]:
# Check a plot of energy as a function of step to see if our run is converging
get_convergence_plots(step_dict=relax_dict,sim_name='ionic relax')

### Run a variable cell relaxation of our structure

In [ ]:
# Create a variable-cell relaxation sim
make_sim("vcrelax", struct,
         control={'pseudo_dir':'./',
                  'calculation':'vc-relax',
                  'outdir':'./',
                  'tstress':True},
         cell={'press':relax_dict['pressure'][-1]},
         system={'ecutwfc':50},
         kpoints_grid=[3,3,3])
# Run vc-relax simulation
run_sim("vcrelax", struct)

In [ ]:
# Extract final energy and pressure
vcrelax_dict = get_qe_outputs('vcrelax.stdout')
final_energy = vcrelax_dict['ionic_energies'][-1]
vcrelaxed_struct = vcrelax_dict['structures']
print(f"{final_energy} Ry\n{vcrelax_dict['pressure'][-1]} kbar")

In [ ]:
# Check a plot of energy as a function of step to see if our run is converging
get_convergence_plots(step_dict=vcrelax_dict,sim_name='vc-relax')